In [2]:
import cv2
import time
import FaceNormalizationUtils as faceutils
import pickle
import numpy as np
import os
from scipy import stats
# mode and so on
from collections import Counter

#MTCNN face detector
from mtcnn.mtcnn import MTCNN
#deepface
from deepface import DeepFace
#from deepface.commons import functions

from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.neighbors import NearestNeighbors

In [17]:
def calc_embs(X, batch_size=2):
    norm_images = prewhiten(X)
    pd = []
    for start in range(0, len(norm_images), batch_size):
        pd.append(model.predict_on_batch(norm_images[start:start+batch_size]))
        
    return l2_normalize(np.concatenate(pd))

def calc_embs_v2(X):
    norm_images = prewhiten(X)
    pd = []
    for start in range(0, len(norm_images)):
        embedding_objs = DeepFace.represent(img_path = norm_images[start],model_name  = "Facenet", enforce_detection = False) 
        pd.append(embedding_objs[0]["embedding"] )
    return l2_normalize(np.concatenate(pd))

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

# Designed for UTKFace as filename contains biometric information
def TrainEmbedingsUTKFace(folders,outputfile):
    nimgs = 0
    Xorig = []
    X = []
    Gender = []
    Age = []
    Etnia = []
    
    for directory in folders:
        #print(directory)
        for path, subdirs, files in os.walk(directory):
            for name in files:
                #print(name)
                if name.endswith(".jpg") and nimgs < 2000:
                #if name.endswith(".png"): # and nimgs < 3000:
                    img_path = os.path.join(path, name)
                    #print(img_path)

                    image = cv2.imread(img_path)

                    if (type(image) is np.ndarray):
                        #print(img_path)
                        # Search face 
                        values = DetectLargestFaceEyesMTCNN(image)
                        if values is not None:
                            #print(nimgs)
                            face, eyes, shape = values

                            # draws container
                            [x, y, w, h] = face
                            if x > -1:
                                # Eyes
                                [lex, ley, rex, rey] = eyes
                                if lex > -1:
                                    
                                    B, G, R = cv2.split(image)

                                    # Normalize for Facenet
                                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Bnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Gnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Rnorm = normalizatorHS.normf_image
                                    NormBGRHS = cv2.merge((Bnorm, Gnorm, Rnorm))
                                    #cv2.imshow("Normalized", NormBGRHS)

                                    # Cropping from HS for facenet
                                    # Usa nyoki https://github.com/nyoki-mtl/keras-facenet
                                    NormBGR = NormBGRHS[35:115, 39:119, :]

                                    # SOft biometric data are extracted from filename
                                    if NormBGR is not None:
                                        nimgs = nimgs + 1
                                        #print(nimgs)

                                        fsub = name.find('_')
                                        sage = name[:fsub]
                                        #print(sage)
                                        sub = name[fsub + 1:]
                                        fsub = sub.find('_')
                                        sgender = sub[:fsub]
                                        sub = sub[fsub + 1:]
                                        fsub = sub.find('_')
                                        setnia = sub[:fsub]

                                        # print(name)
                                        # print(sage)
                                        # print(sgender)
                                        # print(setnia)
                                        # print(nimgs)

                                        # Facenet kearas expects 160x160
                                        #imaged = cv2.resize(NormBGR, (160, 160))
                                        
                                        # Obtiene embeddings
                                        imaged = cv2.resize(NormBGR, dim, interpolation = cv2.INTER_AREA)
                                        
                                        # Mantengo originales para mostrar parecido
                                        imager = cv2.resize(image, (200, 200), interpolation=cv2.INTER_AREA)
                                        Xorig.append(imager)
                                        # Facenet
                                        X.append(imaged)
                                        Gender.append(sgender)
                                        Age.append(sage)
                                        Etnia.append(setnia)
    
    if nimgs > 0:
        # Compute embeddings 
        embs = calc_embs_v2(np.array(X))

        print("Salvando embs")
        fid = open(outputfile, "wb")

        pickle.dump([nimgs, X, embs, Age, Gender, Etnia], fid)
        print(embs.shape)
        fid.close()


In [20]:
def GetSimilarNN(X,idxsimilar, nbrs, embs, Title):
    print(embs)
    # Gets nearest neighbors
    distance, indices = nbrs.kneighbors(embs)
    minp = indices[0][0]

    # Keeps copy of last frames NN
    idxsimilar.extend(indices[0])

    # Remove old ones when more than nframes accumulated
    if len(idxsimilar) > kvecinos * nframeskvecinos:
        idxsimilar = idxsimilar[kvecinos:]

    # print('Lista con ' + str(len(idxsimilar)) + ' ' + str(idxsimilar) + '\n')

    # enough history
    if len(idxsimilar) > kvecinos:
        minp = GetClosesetMode(idxsimilar)

    # Larger for visualization
    imageS = cv2.resize(X[minp], (320, 320))

    # Soft biometrics labels
    if len(idxsimilar) > 0:
        modesG = stats.mode([Gender[int(i)] for i in idxsimilar])
        modesE = stats.mode([Etnia[int(i)] for i in idxsimilar])
        edad = np.array([int(Age[int(i)]) for i in idxsimilar]).mean()

        if modesG[0][0] == '0':
            gen = 'M'
        else:
            gen = 'F'

        if modesE[0][0] == '0':
            et = 'B'
        elif modesE[0][0] == '1':
            et = 'N'
        elif modesE[0][0] == '2':
            et = 'A'
        elif modesE[0][0] == '3':
            et = 'H'
        else:
            et = 'L'

        cv2.putText(imageS, '%s %s %d (%s)' % (gen, et, int(edad), Age[minp]), (10, 30), font, 0.5, (255, 255, 255), 2,
                    cv2.LINE_AA)

    cv2.imshow(Title, imageS)

    return idxsimilar

def getLargestMTCNNBB(objects):
        if len(objects) < 1:
            return -1
        elif len(objects) == 1:
            return 0
        else:
            areas = [ (det['box'][2]*det['box'][3]) for det in objects ]
            return np.argmax(areas)
        
def DetectLargestFaceEyesMTCNN(img):
    global detectormtcnn
    
    results = detectormtcnn.detect_faces(img)

    if not results is None:
        index = getLargestMTCNNBB(results)

        if len(results) < 1:
            return None

        # laergest face
        face_info = results[index]

        #print(face_info)

        [x, y, w, h] = face_info['box']
        le = face_info['keypoints']['left_eye']
        re = face_info['keypoints']['right_eye']

        return [x,y,w,h], [le[0], le[1], re[0], re[1]], [face_info['keypoints']['left_eye'], face_info['keypoints']['right_eye'],
                      face_info['keypoints']['nose'], face_info['keypoints']['mouth_left'],
                      face_info['keypoints']['mouth_right']]
    else:
        return None
    
def ResetDetectionCounters():
    global nconsecutivenodetected, nconsecutivedetected, idxsimilarFN

    nconsecutivenodetected = nconsecutivenodetected + 1
    if nconsecutivenodetected > 10:
        nconsecutivedetected = 0
        idxsimilarFN = []
        
def GetClosesetMode(list):
    # Get occurrences list
    occ = Counter(list)

    # Get the mode, if multiple modes present, gets the clostest observing position among beigbors
    prima = 0
    maxpun = 0
    for neighbor, count in occ.most_common(10):
        if prima == 0:
            prima = 1
            mode = count
        else:
            if count < mode:
                break

        #print('%s: %7d' % (neighbor, count))

        # using enumerate()
        # to find indices for 3
        neighbor_pos = [i for i, value in enumerate(list) if value == neighbor]
        # printing resultant list
        #print("New indices list : " + str(neighbor_pos))

        pun = 0
        for pos in neighbor_pos:
            pun = pun + kvecinos - (pos % kvecinos)
        #print(pun)
        if pun > maxpun:
            maxpun = pun
            closest_neighbor = neighbor

    return closest_neighbor

In [22]:
# Face detector
detectormtcnn = MTCNN()
# Normalization utilities
normalizatorHS = faceutils.Normalization()

# Embeddings deepface
model = DeepFace.build_model("Facenet")
dim = model.input_shape

# 1 to create a new appearance dataset
creadataset = 0
if creadataset == 1:
    print("Create model")
    #Dataset folders
    folders = ["D:/Datasets/Caras/UTKFace/part1", "D:/Datasets/Caras/UTKFace/part2",
               "D:/Datasets/Caras/UTKFace/part3"]
    TrainEmbedingsUTKFace(folders,"D:/FACEScode_models/UTKFace_DLIB_Nyoki_deepfaceSP2021.obj")

#Load appearance dataset
print("Reading model")
#MODIFICAR EN BASE A TU RUTA
#fid = open("C:/Users/otsed/Desktop/Docencia/VC/FACEScode_models/UTKFace_DLIB_Nyoki.obj", "rb") 
fid = open("C:/Users/otsed/Desktop/Docencia/VC/FACEScode_models/UTKFace_DLIB_Nyoki_deepfaceSP2021.obj", "rb") #Modelo reducido
#fid = open("E:/FACEScode_models/UTKFace_DLIB_Nyoki.obj", "rb") # Entrenado similar a normalización Nyoki con 20k
#fid = open("D:/FACEScode_models/UTKFace_DLIB_Nyoki_deepfaceSP2021.obj", "rb") # Entrenado similar a normalización Nyoki con 2k
nimgs, X_FN, embsFN, Age, Gender, Etnia = pickle.load(fid)

print(len(embsFN))
print(embsFN[0])

# Tree for KNN search
kvecinos = 10
nframeskvecinos = 5
nbrsFN = NearestNeighbors(n_neighbors=kvecinos).fit(embsFN)

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection, check unitl one is located
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(2)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 2')
    else:
        print('Camera 1')
else:
    print('Camera 2')
    
#Set camera resolution
cap.set(3,640);
cap.set(4,480);

# Initializations
debug = 0
nconsecutivedetected = 0
nconsecutivenodetected = 0
idxsimilarFN = []

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization split channels
    B, G, R = cv2.split(frame)

    # Search face 
    values = DetectLargestFaceEyesMTCNN(frame)
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:                
                nconsecutivedetected = nconsecutivedetected + 1  #11?
                nconsecutivenodetected = 0
                    
                # Show detected facial elements
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)

                # HS normalization for facenet
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGRHS = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGRHS)
                    
                # Cropping from HS for facenet
                # De HS a lo proximadamente usa nyok https://github.com/nyoki-mtl/keras-facenet
                NormBGR = NormBGRHS[35:115, 39:119, :]
                cv2.imshow("Normalizedc", NormBGR)
                 
                # Obtiene embeddings
                img1 = cv2.resize(NormBGR, dim, interpolation = cv2.INTER_AREA)
                #embs = model.predict(img1[None,...])
                embs = calc_embs_v2(np.array([img1]))
                

                print(len(X_FN))
                print(embs.shape)
                idxsimilarFN = GetSimilarNN(X_FN,idxsimilarFN, nbrsFN, embs.reshape(1, -1), 'FN Me recuerdas a ...')
                
            else:
                ResetDetectionCounters()
        else:
            ResetDetectionCounters()
    else:
        ResetDetectionCounters()

    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(5)
    if tec & tec == 27:  # Esc
        break  

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

Reading model
2000
[ 0.19310383  0.0031643   0.12478592 -0.20989816  0.03106969  0.1003861
  0.03888208  0.10858168  0.04909337  0.06012051  0.01520464  0.01530008
  0.03777692 -0.04863881 -0.01468688 -0.11398428  0.06734373  0.08293412
  0.01901399  0.0459023   0.12020404 -0.11268966 -0.01748156  0.04482151
  0.04064983 -0.1285081  -0.06753954  0.10069403 -0.1781441  -0.05199728
 -0.02251568  0.13836882 -0.13299906  0.06738377 -0.07669103 -0.12358224
  0.03400111 -0.00520453 -0.00024235  0.09055389  0.03688106 -0.07935634
  0.13672787 -0.03770848 -0.0936017  -0.02356653  0.05163802 -0.08694541
  0.04869903 -0.08455293 -0.15029472  0.05402508  0.05865078 -0.13785231
  0.00127104  0.00553483  0.03424353 -0.0839912   0.00178772  0.09450344
  0.03688648  0.15182371 -0.02889229  0.18463968  0.13324045 -0.05565208
  0.11443888  0.01673327  0.11453298 -0.02560252 -0.02619596  0.06534784
  0.04913598  0.00683489  0.08538314  0.01496218 -0.08107303 -0.16824494
 -0.08475927 -0.00872247  0.12587

ValueError: Expected 2D array, got 1D array instead:
array=[-0.04256672 -0.11473004 -0.02355625 -0.12135403  0.09868719  0.03064655
 -0.01051323 -0.05323723 -0.00813854  0.05281385  0.06169069 -0.08097452
  0.0390095  -0.08307895 -0.0248801   0.11976844  0.0880751  -0.08933508
  0.07114759 -0.14221126  0.04878055  0.04171614 -0.04691983 -0.00562779
  0.06325153 -0.0256941   0.13162512  0.11859584 -0.03221897 -0.00493992
 -0.06041448  0.10458144  0.00667513  0.01054465  0.13789662 -0.12098879
  0.04699454  0.03169855  0.12985136  0.02826554  0.06495967 -0.00139198
 -0.02722496 -0.04632006 -0.11827609 -0.1461756   0.0060706  -0.0544685
  0.14308167  0.07179418 -0.13272775  0.08077845  0.057591   -0.11462518
  0.03224185  0.03295977  0.14364871  0.01466678 -0.08844637 -0.24410139
 -0.11754058 -0.0944346  -0.01675603  0.19309601 -0.08529556  0.13027069
  0.08135856  0.01502358  0.07589525 -0.12541221 -0.02316528 -0.11529614
  0.05349674 -0.11267369 -0.01267495 -0.0273954   0.00678179  0.09